# Exploring Foursquare
Updated, again, 05/31/19
* How much data can I get in a day?
* What categories should I examine and how?
* Will "shrinking box" exploration work?
* Does Folium have a square marker (try in other notebook?)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
# @hidden cell
# Foursquare information

CLIENT_ID = 'CWCPNAVXDH3TI1BGS4VED4ANSUKEFGHBA4511GRPYPKPNJRD' # your Foursquare ID
CLIENT_SECRET = 'ALMFEYIENSPH3RV3TQB1NGWTKTANVJ5QTHAVZ5B1GRJWIP21' # your Foursquare Secret

In [0]:
# Foursquare query information that will be constant during this project
VERSION = '20190427' # Foursquare API version
LIMIT = 120

In [0]:
# define function that extracts the category of the venue
# used in Foursquare section

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
# define function to extract Foursquare information given
# a bounding box and a list of categories

def getVenuesByBox(sw, ne,    # the southwest & northeast box corners, tuples
                   codes      # Foursquare venue codes, list of strings
                  ):
  
  # define the search URL
  code_string = ", ".join(codes)  # comma-separated text string
  
  url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
      f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={sw[0]},{sw[1]}' + \
      f'&ne={ne[0]},{ne[1]}' + \
      f'&categoryId={code_string}&limit={LIMIT}'
  
  # submit the url and capture the returned text
  venues = requests.get(url).json()["response"]['groups'][0]['items']
  
  venues = json_normalize(venues) # flatten JSON

  # filter columns
  filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode','venue.location.lat','venue.location.lng', 'venue.location.state']
  enues = venues.loc[:, filtered_columns]

  # filter the category for each row
  venues['venue.categories'] = venues.apply(get_category_type, axis=1)

  # clean columns
  venues.columns = [col.split(".")[-1] for col in venues.columns]
  
  return venues
  
  

In [10]:
asian = ['4bf58dd8d48988d142941735']
fsw = (33.946964, -84.150187)  
fne = (33.971041, -84.121381)


getVenuesByBox(fsw, fne, asian)

,count,items,referralId,categories,id,name,prefix,sizes,name,url,id,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,name,count,groups,id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b2d1a43f964a52056ce24e3-0,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4b2d1a43f964a52056ce24e3,3675 Satellite Blvd #750,US,Duluth,United States,and Pleasant Hill Rd,[3675 Satellite Blvd #750 (and Pleasant Hill R...,"[{'label': 'display', 'lat': 33.96012236013812...",33.960122,-84.136158,NaN,30096,GA,Saigon Cafe Duluth,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bce7684b6c49c7410529791-1,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4bce7684b6c49c7410529791,3040 Steve Reynolds Blvd,US,Duluth,United States,at Old Norcross Rd,[3040 Steve Reynolds Blvd (at Old Norcross Rd)...,"[{'label': 'display', 'lat': 33.96387561289116...",33.963876,-84.141964,NaN,30096,GA,Seo Ra Beol Restaurant,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b5e4548f964a520fe8629e3-2,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4b5e4548f964a520fe8629e3,2174 Pleasant Hill Rd Ste 101,US,Duluth,United States,NaN,"[2174 Pleasant Hill Rd Ste 101, Duluth, GA 300...","[{'label': 'display', 'lat': 33.95982157115721...",33.959822,-84.134848,NaN,30096,GA,Mini Hot Pot 2,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4aad3ed6f964a520385f20e3-3,Supermarket,NaN,NaN,NaN,NaN,NaN,NaN,4aad3ed6f964a520385f20e3,1630 Pleasant Hill Rd,US,Duluth,United States,NaN,"[1630 Pleasant Hill Rd, Duluth, GA 30096, Unit...","[{'label': 'display', 'lat': 33.94716033695705...",33.947160,-84.123698,NaN,30096,GA,Assi Plaza,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4aea12d9f964a52020b921e3-4,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4aea12d9f964a52020b921e3,2570 Pleasant Hill Rd #101,US,Duluth,United States,Pleasant Hill Road,[2570 Pleasant Hill Rd #101 (Pleasant Hill Roa...,"[{'label': 'display', 'lat': 33.96918010417987...",33.969180,-84.145040,NaN,30096,GA,SunO Dessert,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c0d94d2b1b676b062dde086-5,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4c0d94d2b1b676b062dde086,3230 Steve Reynolds Blvd,US,Duluth,United States,NaN,"[3230 Steve Reynolds Blvd, Duluth, GA 30096, U...","[{'label': 'display', 'lat': 33.95785361845068...",33.957854,-84.140973,NaN,30096,GA,Dan Moo Ji,0,[],NaN
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a3193caf964a520f6991fe3-6,Asian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4a3193caf964a520f6991fe3,2131 Pleasant Hill Rd,US,Duluth,United States,NaN,"[2131 Pleasant Hill Rd, Duluth, GA 30096, Unit...","[{'label': 'display', 'lat': 33.95627389003173...",33.956274,-84.134720,NaN,30096,GA,Ming's BBQ,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b734ccef964a52027a72de3-7,Chinese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,4b734ccef964a52027a72de3,2060 Pleasant Hill Rd,US,Duluth,United States,NaN,"[2060 Pleasant Hill Rd, Duluth, GA 30096, Unit...","[{'label': 'display', 'lat': 33.95593298323567...",33.955933,-84.131833,NaN,30096,GA,Panda Express,0,[],NaN
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c12850377cea593f433cd60-8,Supermarket,NaN,NaN,NaN,NaN,NaN,NaN,4c12850377cea593f433cd60,2300 Pleasant Hill Rd,US,Duluth,United States,NaN,"[2300 Pleasant Hill Rd, Duluth, GA 30096, Unit...","[{'label': 'display', 'lat': 33.96353550483943...",33.963536,-84.135243,NaN,30096,GA,Great Wall Supermarket (大中華),0,[],NaN
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bfc4448e05e0f47ba6dcfa8-9,Coffee Shop,NaN,NaN,NaN,NaN,NaN,NaN,4bfc4448e05e0f47ba6dcfa8,3182 Steve Reynolds Blvd Ste 110,US,Duluth,United States,Old Norcross Road,[3182 Steve Reynolds Blvd Ste 110 (Old Norcros...,"[{'label': 'display', 'lat': 33.95820504713777...",33.958205,-84.141408,NaN,30096,GA,7 Coffee,0,[],NaN


In [0]:
# Location for sample query - basically my neighborhood in Duluth

super_h = (33.969303, -84.143626)

# corners for Foursquare bounding box search


albany_ne = (32.114878, -83.489320)
albany_sw = (31.517953, -84.222243)

# Foursquare categories

# Korean restaurant (sub category of above) '4bf58dd8d48988d113941735'
# Indian restaurant (not sub category of above) '4bf58dd8d48988d10f941735'

In [0]:
# Define search url and test.

# fsw, fne = albany_sw, albany_ne



url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
      f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={fsw[0]},{fsw[1]}' + \
      f'&ne={fne[0]},{fne[1]}' + \
      f'&categoryId={asian}' # &limit={LIMIT}'    

results = requests.get(url).json()

In [12]:
# Now convert to dataframe and examine. 

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode','venue.location.lat','venue.location.lng', 'venue.location.state']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.sort_values(by=['categories']).head(50)

KeyError: ignored

In [0]:
print(venues)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b2d1a43f964a52056ce24e3', 'name': 'Saigon Cafe Duluth', 'location': {'address': '3675 Satellite Blvd #750', 'crossStreet': 'and Pleasant Hill Rd', 'lat': 33.96012236013812, 'lng': -84.13615842313993, 'labeledLatLngs': [{'label': 'display', 'lat': 33.96012236013812, 'lng': -84.13615842313993}], 'postalCode': '30096', 'cc': 'US', 'city': 'Duluth', 'state': 'GA', 'country': 'United States', 'formattedAddress': ['3675 Satellite Blvd #750 (and Pleasant Hill Rd)', 'Duluth, GA 30096', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d142941735', 'name': 'Asian Restaurant', 'pluralName': 'Asian Restaurants', 'shortName': 'Asian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b2d1a43f964a52056ce24e3-0'}, {'reasons':

# Data structures needed

    df_box: 
    - sw, ne
    - number of clients
    
    df_client: 
    - Name
    - lat & long
    - categories
    - postal code (?)
    - state

# Shrinking Box Pseudo Code


        
    def shrink(sw, ne, codes)
        df_results = query 4square(sw, ne, codes)
        
        calculate num_clients_ga
        
        if num_clients_ga = 0:
            add current box (sw, ne, num_clients_ga) to df_box
            return
            
        if number of results <100:
            drop clients not in ga
            add current box (sw, ne, num_clients_ga) to df_box
            append Georgia clients to df_client
            return
        
        # case where number of raw results >=100 and at least 1 in GA
        location_set = define 4 tupples of sw&ne locations 
        for box in location_set:
            shrink(box[0, box[1], codes)
            
        return

        
        
        
        

# Foursquare Information

Free plan includes:
* 2 queries per second
* 95,000 regular calls per day
* 500 premium calls per day

Attribution is **required**: Whenever Foursquare data is showing on the screen, attribution to Foursquare must be visible simultaneously or before the Foursquare data is shown.



   